In [1]:
import cv2
import mediapipe as mp
import os
from pathlib import Path
from tqdm import tqdm

In [2]:
# Direktori Input dan Output
RAW_TRAIN = Path("../data/Train")
PROC_TRAIN = Path("../data/processed/Train")

# Inisialisasi MediaPipe Face Detection
mp_face_detection = mp.solutions.face_detection
mp_drawing = mp.solutions.drawing_utils

In [3]:
# Fungsi untuk mendeteksi dan memotong wajah
def detect_and_crop(image_path, target_size=(224, 224)):
    img = cv2.imread(str(image_path))
    
    if img is None:
        return None
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    
    with mp_face_detection.FaceDetection(min_detection_confidence=0.2) as face_detection:
        results = face_detection.process(img_rgb)
        
        if results.detections:
            face = results.detections[0].location_data.relative_bounding_box
            h, w, _ = img.shape
            x1, y1, w1, h1 = int(face.xmin * w), int(face.ymin * h), int(face.width * w), int(face.height * h)
            
            cropped_face = img[y1:y1 + h1, x1:x1 + w1]
            cropped_face = cv2.resize(cropped_face, target_size)
            
            return cropped_face
        else:
            h, w, _ = img.shape
            center_x, center_y = w // 2, h // 2
            size = min(w, h, target_size[0], target_size[1])
            x1 = max(center_x - size // 2, 0)
            y1 = max(center_y - size // 2, 0)
            cropped_face = img[y1:y1 + size, x1:x1 + size]
            
            return cropped_face

In [4]:
# Fungsi untuk memproses seluruh folder gambar
def process_folder(input_dir, output_dir):
    for class_name in os.listdir(input_dir):
        class_path = input_dir / class_name
        if class_path.is_dir():
            output_class_path = output_dir / class_name
            output_class_path.mkdir(parents=True, exist_ok=True)
            
            for img_file in os.listdir(class_path):
                img_path = class_path / img_file
                out_img_path = output_class_path / img_file
                
                face = detect_and_crop(img_path)
                
                if face is not None:
                    cv2.imwrite(str(out_img_path), face)

In [5]:
# Proses gambar di folder training
process_folder(RAW_TRAIN, PROC_TRAIN)

print("Preprocessing selesai!")

Preprocessing selesai!
